In [5]:
import pandas as pd
import numpy as np
from faker import Faker
from datetime import datetime, timedelta
import random
from math import ceil
from pathlib import Path

In [6]:
# Rebuild only the dependent pieces quickly by re-running the full generator (safer for consistency)
random.seed(8)
np.random.seed(8)
fake_ca = Faker('en_CA')
fake_pl = Faker('pl_PL')
today = datetime.today().date()

In [7]:
# Master (same as before)
warehouses = [
    {"warehouse_code": "VAN", "warehouse_name": "Vancouver DC", "city": "Vancouver", "province": "BC"},
    {"warehouse_code": "TOR", "warehouse_name": "Toronto DC", "city": "Toronto", "province": "ON"},
]
pl_ports = ["Port of Gdańsk", "Port of Gdynia", "Port of Szczecin-Świnoujście"]
ca_ports = ["Port of Vancouver", "Port of Montreal"]
inbound_carriers = ["Maersk", "MSC", "CMA CGM", "Hapag-Lloyd"]
incoterms = ["FOB", "CIF", "DDP", "EXW"]
container_types = ["20GP", "40GP", "40HQ"]
categories = ["Lip Balm", "Face Cream", "Face Wash"]
brands = ["NordGlow", "PureBaltic", "LumiCare", "ArcticAura", "Vistula Beauty"]
scents = ["Mint", "Vanilla", "Rose", "Citrus", "Unscented", "Lavender"]
skincare_notes = ["Hydrating", "SPF 15", "SPF 30", "Vitamin C", "Hyaluronic", "Sensitive Skin"]
upc_options = [6, 12, 24, 36]
b2b_customers = [
    "Walmart Canada","Shoppers Drug Mart","Loblaws","London Drugs","Rexall",
    "Costco Canada","Canadian Tire","Sobeys",
]
parcel_carriers = ["Canada Post", "Purolator", "FedEx", "UPS"]
ltl_carriers = ["Day & Ross", "Loomis", "TForce", "Manitoulin"]
west_provinces = {"BC", "AB", "SK", "MB", "YT", "NT", "NU"}

In [8]:
# Items
num_items = 30
items = []
for i in range(1, num_items + 1):
    cat = random.choice(categories)
    brand = random.choice(brands)
    if cat == "Lip Balm":
        name = f"{brand} {random.choice(scents)} {cat}"
    else:
        name = f"{brand} {random.choice(skincare_notes)} {cat}"
    items.append({
        "Item_ID": f"COS-{i:04d}",
        "Item_Name": name,
        "Category": cat,
        "Brand": brand,
        "Country_of_Origin": "Poland",
        "Units_Per_Carton": random.choice(upc_options),
        "Unit_Weight_kg": round(random.uniform(0.03, 0.35), 3),
        "Unit_Volume_cbm": round(random.uniform(0.00015, 0.0012), 6),
        "Standard_Cost_CAD": round(random.uniform(1.2, 6.0), 2),
        "MSRP_CAD": 0.0  # will fill below
    })
item_master = pd.DataFrame(items)
item_master["MSRP_CAD"] = (item_master["Standard_Cost_CAD"] * np.random.uniform(2.0, 3.5, len(item_master))).round(2)
price_map = dict(zip(item_master["Item_ID"], item_master["MSRP_CAD"]))

In [9]:
# POs
num_pos = 120
suppliers = [f"{fake_pl.company()} Sp. z o.o." for _ in range(12)]
po_start = today - timedelta(days=180)
po_records = []
for i in range(1, num_pos + 1):
    po_id = f"PO-{2025}-{i:04d}"
    po_date = po_start + timedelta(days=random.randint(0, 180))
    item = item_master.sample(1).iloc[0]
    cartons = random.randint(10, 200)
    upc = int(item["Units_Per_Carton"])
    qty_pcs = cartons * upc
    dest_wh = random.choice(["VAN","TOR"])
    planned_ship_date = po_date + timedelta(days=random.randint(1, 14))
    po_records.append({
        "PO_ID": po_id,
        "PO_Date": po_date,
        "Supplier_Name": random.choice(suppliers),
        "Item_ID": item["Item_ID"],
        "Ordered_Qty_Cartons": cartons,
        "Ordered_Qty_Pcs": qty_pcs,
        "Unit_Cost_CAD": round(float(item["Standard_Cost_CAD"]) * random.uniform(0.95, 1.10), 2),
        "Warehouse_Destination": dest_wh,
        "Planned_Ship_Date": planned_ship_date
    })
inbound_purchase = pd.DataFrame(po_records)

In [10]:
# Inbound Shipping (1:1 with PO)
ship_records = []
for _, row in inbound_purchase.iterrows():
    discharge = "Port of Vancouver" if row["Warehouse_Destination"] == "VAN" else "Port of Montreal"
    etd = row["Planned_Ship_Date"]
    transit = random.randint(18, 42)
    eta = etd + timedelta(days=transit)
    delay_days = random.choice([-3, -2, -1, 0, 0, 0, 1, 2, 4, 7])
    ata = eta + timedelta(days=delay_days)
    ship_records.append({
        "Inbound_Shipment_ID": f"IM-{row['PO_ID'].split('-')[-1]}",
        "PO_ID": row["PO_ID"],
        "Item_ID": row["Item_ID"],
        "Port_of_Loading": random.choice(pl_ports),
        "Port_of_Discharge": discharge,
        "Carrier": random.choice(["Maersk","MSC","CMA CGM","Hapag-Lloyd"]),
        "Incoterms": random.choice(["FOB","CIF","DDP","EXW"]),
        "Container_Type": random.choice(["20GP","40GP","40HQ"]),
        "Container_Number": f"MSCU{random.randint(1000000,9999999)}",
        "Bill_of_Lading": f"BL{random.randint(100000,999999)}",
        "ETD": etd,
        "ETA": eta,
        "ATD": etd + timedelta(days=random.randint(0,2)),
        "ATA": ata,
        "Transit_Days_Expected": transit,
        "Delay_Days": delay_days,
        "Warehouse_Destination": row["Warehouse_Destination"],
        "Received_Qty_Pcs": row["Ordered_Qty_Pcs"] if ata <= today else 0
    })
inbound_shipping = pd.DataFrame(ship_records)

In [11]:
# Orders & Outbound Shipping
def random_province():
    provinces = ["ON","QC","BC","AB","MB","SK","NS","NB","NL","PE","YT","NT","NU"]
    weights = [0.38,0.23,0.13,0.12,0.04,0.03,0.025,0.02,0.01,0.005,0.003,0.003,0.002]
    return random.choices(provinces, weights=weights, k=1)[0]

def route_wh(prov):
    return "VAN" if prov in west_provinces else "TOR"

orders = []
shipments_out = []
order_counter = 1

# B2C
for _ in range(1200):
    item = item_master.sample(1).iloc[0]
    prov = random_province()
    wh = route_wh(prov)
    order_id = f"SO-{2025}-{order_counter:06d}"
    order_counter += 1
    order_date = today - timedelta(days=random.randint(0, 90))
    qty_each = random.randint(1, 5)
    status = random.choice(["Shipped","Shipped","Shipped","Cancelled","Pending"])
    orders.append({
        "Order_ID": order_id,
        "Order_Date": order_date,
        "Channel": "B2C",
        "Customer_Name": "Ecommerce-CA",
        "Customer_Type": "Consumer",
        "Warehouse": wh,
        "Province": prov,
        "Item_ID": item["Item_ID"],
        "Order_UOM": "Each",
        "Ordered_Qty_Pcs": qty_each,
        "Unit_Sell_Price_CAD": round(price_map[item["Item_ID"]] * random.uniform(0.7, 1.1), 2),
        "Order_Status": status
    })
    if status == "Shipped":
        ship_date = order_date + timedelta(days=random.randint(0,2))
        delivery = ship_date + timedelta(days=random.randint(1,5))
        carrier = random.choice(["Canada Post","Purolator","FedEx","UPS"])
        shipments_out.append({
            "Outbound_Shipment_ID": f"OUT-{order_id.split('-')[-1]}",
            "Order_ID": order_id,
            "Warehouse": wh,
            "Item_ID": item["Item_ID"],              # <-- FIX
            "Carrier": carrier,
            "Service_Level": random.choice(["Ground","Express","Priority"]),
            "Ship_Date": ship_date,
            "Delivery_Date": delivery,
            "Shipped_Qty_Pcs": qty_each,
            "Tracking_Number": f"{carrier[:3].upper()}{random.randint(1000000000,9999999999)}",
            "Freight_Cost_CAD": round(random.uniform(7.5, 18.0), 2),
            "Province": prov
        })

# B2B
for _ in range(300):
    item = item_master.sample(1).iloc[0]
    customer = random.choice(b2b_customers)
    prov = random.choice(["ON","QC","BC","AB","MB","SK","NS","NB"])
    wh = route_wh(prov)
    order_id = f"SO-{2025}-{order_counter:06d}"
    order_counter += 1
    order_date = today - timedelta(days=random.randint(0, 90))
    cartons = random.randint(1, 50)
    qty_pcs = cartons * int(item["Units_Per_Carton"])
    status = random.choice(["Shipped","Shipped","Shipped","Partially Shipped","Pending"])
    orders.append({
        "Order_ID": order_id,
        "Order_Date": order_date,
        "Channel": "B2B",
        "Customer_Name": customer,
        "Customer_Type": "Retailer",
        "Warehouse": wh,
        "Province": prov,
        "Item_ID": item["Item_ID"],
        "Order_UOM": "Carton",
        "Ordered_Qty_Pcs": qty_pcs,
        "Unit_Sell_Price_CAD": round(price_map[item["Item_ID"]] * random.uniform(0.45, 0.65), 2),
        "Order_Status": status
    })
    if status in {"Shipped","Partially Shipped"}:
        ship_date = order_date + timedelta(days=random.randint(1,5))
        delivery = ship_date + timedelta(days=random.randint(2,8))
        carrier = random.choice(["Day & Ross","Loomis","TForce","Manitoulin"])
        shipped_qty = qty_pcs if status == "Shipped" else int(qty_pcs * random.uniform(0.6,0.9))
        shipments_out.append({
            "Outbound_Shipment_ID": f"OUT-{order_id.split('-')[-1]}",
            "Order_ID": order_id,
            "Warehouse": wh,
            "Item_ID": item["Item_ID"],              # <-- FIX
            "Carrier": carrier,
            "Service_Level": "LTL",
            "Ship_Date": ship_date,
            "Delivery_Date": delivery,
            "Shipped_Qty_Pcs": shipped_qty,
            "Tracking_Number": f"{carrier[:3].upper()}{random.randint(100000000,999999999)}",
            "Freight_Cost_CAD": round(random.uniform(85.0, 450.0), 2),
            "Province": prov
        })

outbound_orders = pd.DataFrame(orders)
outbound_shipping = pd.DataFrame(shipments_out)

In [13]:
# Inventory snapshot (recompute)
base_stock = {(wh["warehouse_code"], item_id): random.randint(300, 1200)
              for wh in warehouses for item_id in item_master["Item_ID"]}

receipts = inbound_shipping[inbound_shipping["ATA"] <= pd.Timestamp(today)].groupby(
    ["Warehouse_Destination", "Item_ID"]
)["Received_Qty_Pcs"].sum().to_dict()

shipments = outbound_shipping[outbound_shipping["Ship_Date"] <= pd.Timestamp(today)].groupby(
    ["Warehouse", "Item_ID"]
)["Shipped_Qty_Pcs"].sum().to_dict()

not_arrived = inbound_shipping[inbound_shipping["ATA"] > pd.Timestamp(today)].merge(
    inbound_purchase[["PO_ID", "Ordered_Qty_Pcs"]], on="PO_ID", how="left"
).groupby(["Warehouse_Destination", "Item_ID"])["Ordered_Qty_Pcs"].sum().to_dict()

rows = []
for (wh, item_id), start_qty in base_stock.items():
    rec = receipts.get((wh, item_id), 0)
    shp = shipments.get((wh, item_id), 0)
    on_hand = max(0, start_qty + rec - shp)
    on_ord = not_arrived.get((wh, item_id), 0)
    allocated = max(0, int(on_hand * random.uniform(0.05, 0.25)))
    available = max(0, on_hand - allocated)
    rows.append({
        "Snapshot_Date": today,
        "Warehouse": wh,
        "Item_ID": item_id,
        "On_Hand_Pcs": int(on_hand),
        "Allocated_Pcs": int(allocated),
        "Available_Pcs": int(available),
        "On_Order_Pcs": int(on_ord)
    })
inventory_snapshot = pd.DataFrame(rows)

C:\Users\mycho\anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)
C:\Users\mycho\anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)
C:\Users\mycho\anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable

In [14]:
# Save over previous CSVs
outdir = Path("/mnt/data/cosmetics_dataset")
outdir.mkdir(parents=True, exist_ok=True)
paths = {}
def save(df, name):
    p = outdir / f"{name}.csv"
    df.to_csv(p, index=False)
    paths[name] = str(p)

save(item_master, "item_master")
save(inbound_purchase, "inbound_purchase_data")
save(inbound_shipping, "inbound_shipping_data")
save(outbound_orders, "outbound_orders_data")
save(outbound_shipping, "outbound_shipping_data")
save(inventory_snapshot, "inventory_snapshot")

In [18]:
# Zip
import zipfile
zip_path = outdir / "cosmetics_dataset.zip"
with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zf:
    for name, p in paths.items():
        zf.write(p, arcname=f"{name}.csv")

# Show previews in console instead of sandbox-only function
print("\n=== Item Master (preview) ===")
print(item_master.head(20))

print("\n=== Inbound Purchase Data (preview) ===")
print(inbound_purchase.head(20))

print("\n=== Inbound Shipping Data (preview) ===")
print(inbound_shipping.head(20))

print("\n=== Outbound Orders Data (preview) ===")
print(outbound_orders.head(20))

print("\n=== Outbound Shipping Data (preview) ===")
print(outbound_shipping.head(20))

print("\n=== Inventory Snapshot (preview) ===")
print(inventory_snapshot.head(20))

# If you want, also print file paths
print("\nFiles saved:")
for k, v in paths.items():
    print(f"{k}: {v}")
print(f"Zipped dataset: {zip_path}")



=== Item Master (preview) ===
     Item_ID                            Item_Name    Category           Brand  \
0   COS-0001             LumiCare Citrus Lip Balm    Lip Balm        LumiCare   
1   COS-0002       PureBaltic Vitamin C Face Wash   Face Wash      PureBaltic   
2   COS-0003             ArcticAura Mint Lip Balm    Lip Balm      ArcticAura   
3   COS-0004      ArcticAura Vitamin C Face Cream  Face Cream      ArcticAura   
4   COS-0005  Vistula Beauty Vitamin C Face Cream  Face Cream  Vistula Beauty   
5   COS-0006     Vistula Beauty SPF 15 Face Cream  Face Cream  Vistula Beauty   
6   COS-0007          PureBaltic SPF 15 Face Wash   Face Wash      PureBaltic   
7   COS-0008         ArcticAura SPF 15 Face Cream  Face Cream      ArcticAura   
8   COS-0009    LumiCare Sensitive Skin Face Wash   Face Wash        LumiCare   
9   COS-0010        LumiCare Hyaluronic Face Wash   Face Wash        LumiCare   
10  COS-0011          PureBaltic SPF 30 Face Wash   Face Wash      PureBaltic 